In [11]:

%load_ext autoreload

%autoreload 2

from lstar_extraction.LSTM import LSTMNetwork
# from GRU import GRUNetwork
from lstar_extraction.RNNClassifier import RNNClassifier
from lstar_extraction.Training_Functions import mixed_curriculum_train,make_train_set_for_target,make_test_set
from lstar_extraction.Tomita_Grammars import tomita_1, tomita_2, tomita_3, tomita_4, tomita_5, tomita_6, tomita_7, tomita_8, tomita_9







The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# make training set
target = tomita_9

alphabet = "lr"

# alphabet = "abcd"
train_set = make_train_set_for_target(target,alphabet)

# define rnn
rnn = RNNClassifier(alphabet,num_layers=1,hidden_dim=10,RNNClass = LSTMNetwork)


print(train_set)


made train set of size: 615 , of which positive examples: 156
{'': True, 'r': False, 'l': False, 'lr': True, 'rr': False, 'rl': False, 'll': False, 'llr': False, 'rlr': False, 'rrl': False, 'rrr': False, 'lrl': False, 'rll': False, 'lrr': False, 'lll': False, 'lrlr': True, 'llrr': True, 'lllr': False, 'llll': False, 'rllr': False, 'lrll': False, 'rlll': False, 'rrll': False, 'lrrl': False, 'rrrl': False, 'lrrr': False, 'rrlr': False, 'rlrr': False, 'rlrl': False, 'rrrr': False, 'llrl': False, 'rlrlr': False, 'rrllr': False, 'rllll': False, 'llrll': False, 'lrrll': False, 'llrrl': False, 'rrlrr': False, 'rrrrr': False, 'lllrr': False, 'rllrl': False, 'llllr': False, 'lrlrr': False, 'rrlrl': False, 'lrrrl': False, 'rrrlr': False, 'lrllr': False, 'lllll': False, 'lrlrl': False, 'llrrr': False, 'rlrll': False, 'lrlrlr': True, 'llrlrr': True, 'lrllrr': True, 'llrrlr': True, 'lllrrr': True, 'lrrlrl': False, 'rlllll': False, 'rrrrlr': False, 'rlrlrl': False, 'rllrrr': False, 'rllrrl': False, 

In [13]:
# train the model
mixed_curriculum_train(rnn,train_set,stop_threshold = 0.0005)
rnn.renew()  


current average loss is:  0.1551640687450286
current average loss is:  0.061821911684287495
current average loss is:  0.03770749256695791
current average loss is:  0.018202698084535777
current average loss is:  0.010237993098925596
current average loss is:  0.0062980589827870935
current average loss is:  0.004890851664349316
current average loss is:  0.003840804293872864
current average loss is:  0.0031118468540470776
current average loss is:  0.002582856697764823
current average loss is:  0.0008130156211004221
classification loss on last batch was: 0.00045259467478508573


In [17]:
 # generate_test_set


test_set=make_test_set(alphabet)

from test_xRNN import Traces



traces=Traces(rnn, alphabet)
traces.label_from_network(test_set)
traces.write_in_file()

# starting examples
# all_words = sorted(list(train_set.keys()),key=lambda x:len(x))
# pos = next((w for w in all_words if rnn.classify_word(w)==True),None)
# neg = next((w for w in all_words if rnn.classify_word(w)==False),None)
# starting_examples = [w for w in [pos,neg] if not None == w]



made test set of size: 165


In [20]:


# rnn.renew() 

# extract DFA from RNN
from lstar_extraction.Extraction import extract


dfa, ltl = extract(rnn,alphabet,traces, time_limit = 50,initial_split_depth = 10,starting_examples=None)

# print(dfa)
print("\n\nfinal LTL formula:", ltl)
# dfa.draw_nicely(maximum=30) #max size willing to draw



provided counterexamples are: None



start formula depth: 1
cleaning unnecessary files
learned LTL formula: ((l | r) -> (X r))
obs table refinement took 0.562
guided starting equivalence query for DFA of size 4
lstar successful: unrolling seems equivalent to proposed automaton
equivalence checking took: 0.00019199999999841566
overall guided extraction time took: 0.5629649999999913
generated counterexamples were: (format: (counterexample, counterexample generation time))



final LTL formula: ((l | r) -> (X r))
